# Import the necessary packages

In [1]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.text import text_to_word_sequence
from keras.models import load_model
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import csv
import re
import statistics

Using TensorFlow backend.


## Read the csv file

In [2]:
data = pd.read_csv('employee.csv')
columns = data.columns

## Function for plotting the graphs

In [3]:
def plots(pairs,data):
    #reading of files for local function
#     data = pd.read_csv('employee.csv')
    columns = data.columns
    # splitting into x and y
    x = []
    y = []
    colx = pairs[0]
    coly = pairs[1]
    x.append(data[colx])
    y.append(data[coly])
    # reading it element wise so that we can change from numpy array to list format
    final_x = []
    final_y = []
    for j in range(x[0].shape[0]):
        final_x.append(x[0][j])
        final_y.append(y[0][j])
        
    plt.scatter(final_x,final_y)
    plt.title(colx + ' vs ' + coly)
    plt.show()

## Function for restaurant data post processing

In [4]:
# def infer(input_keys, input_token_index):
#     input_texts = []

#     name_text = input_keys['name']
#     eatType_text = input_keys['eatType']
#     food_text = input_keys['food']
#     priceRange_text = input_keys['priceRange']
#     customerRating_text = input_keys['customerRating']
#     area_text = input_keys['area']
#     kidsFriendly_text = input_keys['familyFriendly']
#     near_text = input_keys['near']

#     name_string = 'start_name ' + name_text + ' stop_name' if name_text else 'start_name stop_name'
#     eatType_string = 'start_eattype ' + eatType_text + ' stop_eattype' if eatType_text else 'start_eattype stop_eattype'
#     food_string = 'start_food ' + food_text + ' stop_food' if food_text else 'start_food stop_food'
#     priceRange_string = 'start_pricerange ' + priceRange_text + ' stop_pricerange' if priceRange_text else 'start_pricerange stop_pricerange'
#     customerRating_string = 'start_customerrating ' + customerRating_text + ' stop_customerrating' if customerRating_text else 'start_customerrating stop_customerrating'
#     area_string = 'start_area ' + area_text + ' stop_area' if area_text else 'start_area stop_area'
#     kidsFriendly_string = 'start_kidsfriendly ' + kidsFriendly_text + ' stop_kidsfriendly' if kidsFriendly_text else 'start_kidsfriendly stop_kidsfriendly'
#     near_string = 'start_near ' + near_text + ' stop_near' if near_text else 'start_near stop_near'

#     input_string = ' '.join(
#         [name_string, eatType_string, food_string, priceRange_string, customerRating_string, area_string,
#          kidsFriendly_string, near_string])
#     input_texts.append(input_string)

#     encoder_input_data = np.zeros(
#         (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
#         dtype='float32')

#     for i, (input_text) in enumerate(input_texts):
#         for t, wor in enumerate(input_text.split(" ")):
#             encoder_input_data[i, t, input_token_index[wor]] = 1.

#     return encoder_input_data

## Function for post processing

In [5]:
def infer_graphs(input_keys, input_token_index):
    input_texts = []
    # extracting the key and values from the dictionary
    name = str(input_keys['mean'])
    median_text = str(int(input_keys['median']))
    mode_text = str(input_keys['mode'])
    colx_text = str(input_keys['colx']).lower()
    coly_text = str(input_keys['coly']).lower()
    # Adding start and stop words
    name_string = 'start_name ' + name + ' stop_name' if name else 'start_name stop_name'
    median_string = 'start_median ' + median_text + ' stop_median' if median_text else 'start_median stop_median'
    mode_string = 'start_mode ' + mode_text + ' stop_mode' if mode_text else 'start_mode stop_mode'
    colx_string = 'start_colx ' + colx_text + ' stop_colx' if colx_text else 'start_colx stop_colx'
    coly_string = 'start_coly ' + coly_text + ' stop_coly' if coly_text else 'start_coly stop_coly'
    # appending it to the input texts
    input_string = ' '.join(
        [name_string, median_string, mode_string, colx_string, coly_string])
    input_texts.append(input_string)
    print(input_texts[0])
    # creating the encoder data to be fed to the model
    encoder_input_data = np.zeros(
        (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
        dtype='float32')

    for i, (input_text) in enumerate(input_texts):
        for t, wor in enumerate(input_text.split(" ")):
            encoder_input_data[i, t, input_token_index[wor]] = 1.

    return encoder_input_data

## Decoding the numbers to sentences

In [6]:
def decode_sequence(input_seq, enc_model, dec_model, reverse_target_token_index):
    # print(input_seq)
    states_value = enc_model.predict(input_seq)  # Encode the input
    target_seq = np.zeros((1, 1, num_decoder_tokens))  # Empty target sequence of length 1
    target_seq[0, 0, target_token_index['\t']] = 1.  # Start token for target and update for each new prediction

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = dec_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_wor = reverse_target_token_index[sampled_token_index]
        decoded_sentence += ' ' + sampled_wor

        # Exit condition: either hit max length or find stop token
        if (sampled_wor == '\n' or
                len(decoded_sentence) > max_decoder_seq_length * 2):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

## Training module for deep learning

In [7]:
def train(encoder_input_data, decoder_input_data, decoder_target_data):
    # hyper parmaeters
    batch_size = 64    # Batch size for training.
    epochs = 15     # Number of epochs to train for.
    latent_dim = 256   # Latent dimensionality of the encoding space.
    
    # Build encoder
    encoder_inputs = Input(shape=(None, num_encoder_tokens))      # unique input tokens
    encoder = LSTM(latent_dim, return_state=True)                 # number of neurons
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)

    encoder_states = [state_h, state_c]                           # We discard the outputs
    
    # Build the decoder, using the context of the encoder
    decoder_inputs = Input(shape=(None, num_decoder_tokens))                  # unique output tokens
    decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) 
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(num_decoder_tokens, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model.summary()

    # Run training
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
    # extra parameters can be added for the output metrics such as accuracy and validation losses
    model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
              batch_size=batch_size,
              epochs=epochs)
    model.save('models/master_model_graph.h5')

    # Inference model architecture
    # encoder model
    encoder_model = Model(encoder_inputs, encoder_states)
    encoder_model.summary()
    encoder_model.save('models/encoder_model_graph.h5')
    # decoder model
    decoder_state_input_h = Input(shape=(latent_dim,))
    decoder_state_input_c = Input(shape=(latent_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    decoder_model.summary()
    decoder_model.save('models/decoder_model_graph.h5')

## Statistics calculation for graph summary

In [14]:
def summary(pairs):
    data = pd.read_csv('employee.csv')
    columns = data.columns
    
    x = []
    y = []
    colx = pairs[0]
    coly = pairs[1]
    x.append(data[colx])
    y.append(data[coly])
    
    final_x = []
    final_y = []
    for j in range(x[0].shape[0]):
        final_x.append(x[0][j])
        final_y.append(y[0][j])
    # statistical value calculations for the given pair of inputs
    mean = statistics.mean(final_x)
    median = statistics.median(final_x)
    mode = statistics.mode(final_x)
    
    return mean, median, mode, colx, coly

## Calling out dependent functions for inferencing

In [9]:
def inference(input_token_index, target_token_index,encoder_input_data, pairs):
    mean, median, mode, colx, coly = summary(pairs)
    # reversing the token and word combination
    reverse_input_token_index = dict(
        (i, wor) for wor, i in input_token_index.items())
    reverse_target_token_index = dict(
        (i, wor) for wor, i in target_token_index.items())
    # laoding the encoder and decoder models for inferencing
    enc_mod = load_model('models/encoder_model_graph.h5')
    dec_mod = load_model('models/decoder_model_graph.h5')

    for seq_index in range(1):
        # for statistical data
        input_keys_graph = {'mean': mean, 'median': median, 'mode': mode, 'colx': colx, 'coly': coly}
        input_seq_user = infer_graphs(input_keys_graph, input_token_index)
        decoded_sentence = decode_sequence(input_seq_user, enc_mod, dec_mod, reverse_target_token_index)
        
        print('Input sentence:\n', input_keys_graph)
        print('---------')
        print('Decoded sentence:\n', decoded_sentence)
        
#         # for custom restaurant data
#         input_keys = {'name': 'the mill', 'eatType': 'pub', 'food': 'english', 'priceRange': 'high',
#                       'customerRating': '5',
#                       'area': 'city centre', 'familyFriendly': 'no', 'near': ''}
#         input_seq_user = infer(input_keys, input_token_index)
#         decoded_sentence = decode_sequence(input_seq_user, enc_mod, dec_mod, reverse_target_token_index)

#         print('Input sentence:\n', input_keys)
#         print('---------')
#         print('Decoded sentence:\n', decoded_sentence)

#         # for data out of the training data
#         input_seq = encoder_input_data[seq_index: seq_index + 1]
#         decoded_sentence = decode_sequence(input_seq, enc_mod, dec_mod)

#         print('-')
#         print('Input sentence:\n', input_texts[seq_index])
#         print('Decoded sentence:\n', decoded_sentence)
#         print('-')

## Data prep for restaurant data

In [10]:
# data_path = 'e2e-dataset/trainset.csv'
# input_texts = []
# target_texts = []
# input_vocab = set()
# target_vocab = set()

# with open(data_path, 'r', encoding='utf-8') as f:
#     reader = csv.reader(f)
#     training_set = list(reader)

# for element in training_set[1:]:
#     input_text = element[0]

#     name_text = re.search('(?<=name\[).+?(?=\])', input_text)
#     eatType_text = re.search('(?<=eatType\[).+?(?=\])', input_text)
#     food_text = re.search('(?<=food\[).+?(?=\])', input_text)
#     priceRange_text = re.search('(?<=priceRange\[).+?(?=\])', input_text)
#     customerRating_text = re.search('(?<=customer rating\[).+?(?=\])', input_text)
#     area_text = re.search('(?<=area\[).+?(?=\])', input_text)
#     kidsFriendly_text = re.search('(?<=familyFriendly\[).+?(?=\])', input_text)
#     near_text = re.search('(?<=near\[).+?(?=\])', input_text)

#     name_string = 'start_name ' + name_text.group(0) + ' stop_name' if name_text else 'start_name stop_name'
#     eatType_string = 'start_eatType ' + eatType_text.group(
#         0) + ' stop_eatType' if eatType_text else 'start_eatType stop_eatType'
#     food_string = 'start_food ' + food_text.group(0) + ' stop_food' if food_text else 'start_food stop_food'
#     priceRange_string = 'start_priceRange ' + priceRange_text.group(
#         0) + ' stop_priceRange' if priceRange_text else 'start_priceRange stop_priceRange'
#     customerRating_string = 'start_customerRating ' + customerRating_text.group(
#         0) + ' stop_customerRating' if customerRating_text else 'start_customerRating stop_customerRating'
#     area_string = 'start_area ' + area_text.group(0) + ' stop_area' if area_text else 'start_area stop_area'
#     kidsFriendly_string = 'start_kidsFriendly ' + kidsFriendly_text.group(
#         0) + ' stop_kidsFriendly' if kidsFriendly_text else 'start_kidsFriendly stop_kidsFriendly'
#     near_string = 'start_near ' + near_text.group(0) + ' stop_near' if near_text else 'start_near stop_near'

#     input_string = ' '.join(
#         [name_string, eatType_string, food_string, priceRange_string, customerRating_string, area_string,
#          kidsFriendly_string, near_string])
#     input_texts.append(input_string)

#     target_text = element[1]
#     target_text = '\t ' + target_text + ' \n'
#     target_texts.append(target_text)
#     # print(input_string)
#     # print(target_text)

# input_vocab = set(text_to_word_sequence(" ".join(input_texts), filters='!"#$%&()*+,-./:;<=>?@[\]^`{|}~'))
# target_vocab = set(text_to_word_sequence(" ".join(target_texts), filters='!"#$%&()*+,-./:;<=>?@[\]^`{|}~'))

# input_text_modif = []
# for input_text in input_texts:
#     input_text_modif.append(' '.join(text_to_word_sequence(input_text, filters='!"#$%&()*+,-./:;<=>?@[\]^`{|}~', lower=True)))

# target_text_modif = []
# for target_text in target_texts:
#     target_text_modif.append(' '.join(text_to_word_sequence(target_text, filters='!"#$%&()*+,-./:;<=>?@[\]^`{|}~', lower=True)))

# input_texts = input_text_modif
# target_texts = target_text_modif

# input_vocab = sorted(list(input_vocab))
# target_vocab = sorted(list(target_vocab))
# num_encoder_tokens = len(input_vocab)
# num_decoder_tokens = len(target_vocab)
# max_encoder_seq_length = max([len(txt.split(" ")) for txt in input_texts])
# max_decoder_seq_length = max([len(txt.split(" ")) for txt in target_texts])
# print('Number of samples:', len(input_texts))
# print('Number of unique input tokens:', num_encoder_tokens)
# print('Number of unique output tokens:', num_decoder_tokens)
# print('Max sequence length for inputs:', max_encoder_seq_length)
# print('Max sequence length for outputs:', max_decoder_seq_length)

# input_token_index = dict(
#     [(wor, i) for i, wor in enumerate(input_vocab)])
# target_token_index = dict(
#     [(wor, i) for i, wor in enumerate(target_vocab)])
# # print(input_token_index)

# encoder_input_data = np.zeros(
#     (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
#     dtype='float32')
# decoder_input_data = np.zeros(
#     (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
#     dtype='float32')
# decoder_target_data = np.zeros(
#     (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
#     dtype='float32')

# for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
#     for t, wor in enumerate(input_text.split(" ")):
#         encoder_input_data[i, t, input_token_index[wor]] = 1.
#     for t, wor in enumerate(target_text.split(" ")):
#         decoder_input_data[i, t, target_token_index[wor]] = 1.
#         if t > 0:
#             decoder_target_data[i, t - 1, target_token_index[wor]] = 1.

## Data prep for statistics data

In [11]:
data_path = 'dummy_data_new.csv'
input_texts = []
target_texts = []
input_vocab = set()
target_vocab = set()

# reading the .csv file
with open(data_path, 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    training_set = list(reader)

for element in training_set[1:]:
    input_text = element[0]
    # keyword search through the .csv file
    name = re.search('(?<=mean\[).+?(?=\])', input_text)
    median_text = re.search('(?<=median\[).+?(?=\])', input_text)
    mode_text = re.search('(?<=mode\[).+?(?=\])', input_text)
    colx_text = re.search('(?<=colx\[).+?(?=\])', input_text)
    coly_text = re.search('(?<=coly\[).+?(?=\])', input_text)
    # adding the start and stop words
    name_string = 'start_name ' + name.group(0) + ' stop_name' if name else 'start_name stop_name'
    median_string = 'start_median ' + median_text.group(
        0) + ' stop_median' if median_text else 'start_median stop_median'
    mode_string = 'start_mode ' + mode_text.group(0) + ' stop_mode' if mode_text else 'start_mode stop_mode'
    colx_string = 'start_colx ' + colx_text.group(
        0) + ' stop_colx' if colx_text else 'start_colx stop_colx'
    coly_string = 'start_coly ' + coly_text.group(
        0) + ' stop_coly' if coly_text else 'start_coly stop_coly'
    # adding the processed text to input texts
    input_string = ' '.join(
        [name_string, median_string, mode_string, colx_string, coly_string])
    input_texts.append(input_string)
    # creating the labels for deep learning model
    target_text = element[1]
    target_text = '\t ' + target_text + ' \n'
    target_texts.append(target_text)
    # print(input_string)
    # print(target_text)
# creating the input and output vocab as sets so only unique characters are stored
input_vocab = set(text_to_word_sequence(" ".join(input_texts), filters='!"#$%&()*+,-./:;<=>?@[\]^`{|}~'))
target_vocab = set(text_to_word_sequence(" ".join(target_texts), filters='!"#$%&()*+,-./:;<=>?@[\]^`{|}~'))

input_text_modif = []
for input_text in input_texts:
    input_text_modif.append(' '.join(text_to_word_sequence(input_text, filters='!"#$%&()*+,-./:;<=>?@[\]^`{|}~', lower=True)))

target_text_modif = []
for target_text in target_texts:
    target_text_modif.append(' '.join(text_to_word_sequence(target_text, filters='!"#$%&()*+,-./:;<=>?@[\]^`{|}~', lower=True)))

input_texts = input_text_modif
target_texts = target_text_modif

input_vocab = sorted(list(input_vocab))
target_vocab = sorted(list(target_vocab))
num_encoder_tokens = len(input_vocab)
num_decoder_tokens = len(target_vocab)
max_encoder_seq_length = max([len(txt.split(" ")) for txt in input_texts])
max_decoder_seq_length = max([len(txt.split(" ")) for txt in target_texts])
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)
# creating the tokens for the vocabulary for pre-processing
input_token_index = dict(
    [(wor, i) for i, wor in enumerate(input_vocab)])
target_token_index = dict(
    [(wor, i) for i, wor in enumerate(target_vocab)])
# print(input_token_index)
# creating the input and output tokens for the encoder - decoder model
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, wor in enumerate(input_text.split(" ")):
        encoder_input_data[i, t, input_token_index[wor]] = 1.
    for t, wor in enumerate(target_text.split(" ")):
        decoder_input_data[i, t, target_token_index[wor]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[wor]] = 1.

Number of samples: 1
Number of unique input tokens: 15
Number of unique output tokens: 15
Max sequence length for inputs: 15
Max sequence length for outputs: 16


## Calling the trainig module

In [12]:
train(encoder_input_data, decoder_input_data, decoder_target_data)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 15)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 15)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 278528      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  278528      input_2[0][0]                    
                                                                 lstm_1[0][1]               

## Preparing the widgets for UI

In [17]:
print("RUN THIS CELL TWICE")
# creating menu with them
variables = widgets.Dropdown(options=columns)
variables_1 = widgets.Dropdown(options=columns)
# button, output, function and linkage
butt = widgets.Button(description='Column Y')
butt_1 = widgets.Button(description='Column X')
out = widgets.Output()

pairs = []

def on_butt_clicked_1(_):
    with out:
        clear_output()
#         print('column_x: ' + variables_1.value)
        pairs.append(variables_1.value)

def on_butt_clicked(_):
    with out:
        clear_output()
        print('column_x: ' + variables_1.value)
        print('column_y: ' + variables.value)
        pairs.append(variables.value)
        if len(pairs)>1:
            display(Markdown(plots(pairs, data)))
            display(Markdown(inference(input_token_index, target_token_index, encoder_input_data, pairs)))
            pairs.clear()
        
butt.on_click(on_butt_clicked)
butt_1.on_click(on_butt_clicked_1)
# display
box = widgets.VBox([variables_1, butt_1, variables, butt, out])
box

RUN THIS CELL TWICE
